In [1]:
import pandas as pd
from pymongo import MongoClient
import re
from haversine import haversine
client = MongoClient()
import function as f
import time

In [10]:
places = f.search_places_by_input()

Query: new york 500
{'html_attributions': [], 'next_page_token': 'CoQC-QAAAOQTO2Drnt2w9DSZshTYphWYA5rrBBK2KmZ4aYU2JWDd3MUrptTwXbTNPqPshkBCs8RAWQUG_m-TORsom3RNwUKaW0rU-YldgnX0aQS6xqvVH66keM9M8QbcTdXUt7xzgJoDSaRcPo7FXZ1yQCDZtHY6CUDoreq-WlAdEzX53p3MN0blKO9Y1wY_mvW8NOZThruWHTtdBjqUTlG9yhUPAerOarVLF5X_AKq5zMJuohCgl9T_VNDsXttRazkYm5XdG5YWgMMvpwAkhajPIUaNOtlIM_CVvqEJKknBTwKEZxzE5REgdDnavS7okKD9K4LM6Nj-kGtRig015SY1JpU7iLYSEAN1Cj11T8ZDVowNprWOoAoaFKQ9MSoCZpcxEq34EuetjQ0cT58t', 'results': [{'formatted_address': '500 7th Ave, New York, NY 10018, United States', 'geometry': {'location': {'lat': 40.7532064, 'lng': -73.9893264}, 'viewport': {'northeast': {'lat': 40.75450727989272, 'lng': -73.98786382010728}, 'southwest': {'lat': 40.75180762010728, 'lng': -73.99056347989271}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': 'b19538bbe9815ee879ff53a5ff1b2bdaf69a1d1c', 'name': 'WeWork', 'opening_hours': {'open_now': False}, 'photos': [{'height': 426, 'html_at

KeyError: 'next_page_token'

# Bring documents from Mongodb

In [2]:
db, coll = f.connectCollection('companies','companies')

In [3]:
compas = list(coll.find({"$and":[{"deadpooled_year":None},{"founded_year":{"$gte":1999}},{"$where":"this.offices.length >= 1"},
         {"$or":[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]}]}))

# Filtering and new collections

In [4]:
db, coll1 = f.connectCollection('companies','airports')
db, coll2 = f.connectCollection('companies','preferred')

In [5]:
airports = list(coll1.find({"type": 'large_airport'}))

In [6]:
business = list(coll2.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

Create new collections:

In [7]:
'''
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["companies"]
mycol = mydb["final5"]
'''

'\nmyclient = MongoClient("mongodb://localhost/")\nmydb = myclient["companies"]\nmycol = mydb["final5"]\n'

To import a csv to Mongodb:

In [8]:
'''
mongoimport --db companies --collection kindergarten --type csv --file kindergarten.csv --headerline
'''

'\nmongoimport --db companies --collection kindergarten --type csv --file kindergarten.csv --headerline\n'

# Harversine 

Since the Harversine function takes a couple of coordinates and return the distance in between, it helped to conclude which of the 99 companies have an airport the closest. 

These 99 companies are the preferred after applying the different filters mentioned before. 

In [9]:
name=[]
for bus in business:
    name.append(bus['name'])

In [10]:
lat1=[]
long1=[]
f.getcoord(business,lat1,long1)

(99, 99)

In [11]:
bus=list(zip(lat1,long1))

In [12]:
lat2=[]
long2=[]
f.getcoord(airports,lat2,long2)

(614, 614)

In [13]:
air=list(zip(lat2,long2))

In [14]:
futcol=[]
for i in bus:
    futcol.append(f.harver(i,air))

In [15]:
dfair=pd.DataFrame({"name":name,"long":long1,"lat":lat1,"dis to an airport":futcol})

In [16]:
actuali=list(zip(business,futcol))

In [17]:
for i in actuali:
    value={"$set":{'distance':i[1]}}
    coll1.update_one(i[0],value)

In [18]:
cutoffs = [0,10,20,50,100,300]
km_labels = ['0-10km', '10-20km', '20-50km','50-100km','100-300km']
dfair['range'] = pd.cut(dfair['dis to an airport'], cutoffs, labels = km_labels)
dfair['range'].value_counts()

10-20km      43
0-10km       26
20-50km      24
50-100km      4
100-300km     2
Name: range, dtype: int64

# Google API

In [19]:
lat=[]
long=[]
f.getcoord(business,lat,long)

(99, 99)

In [20]:
lista=[]
busines=list(zip(lat,long))

In [21]:
for i in busines:
    lista.append("{},{}".format(i[0],i[1]))

### API request

In [22]:
lista

['40.757929,-73.985506',
 '33.8171,-111.9035',
 '37.566879,-122.323895',
 '37.798853,-122.398599',
 '34.047312,-118.445243',
 '37.446823,-122.161523',
 '34.0698494,-118.4054182',
 '37.5645375,-122.3254705',
 '37.772323,-122.214897',
 '34.052187,-118.243425',
 '51.51819,-0.208979',
 '34.052187,-118.243425',
 '42.275263,-71.24762',
 '53.544711,-113.515769',
 '54.703145,-1.574948',
 '37.424614,-122.145153',
 '21.303049,-157.78907',
 '39.749671,-104.9998449',
 '59.337864,18.036587',
 '27.913011,-82.702449',
 '40.714056,-73.957854',
 '40.768058,-73.956599',
 '37.424614,-122.145153',
 '21.303049,-157.78907',
 '39.749671,-104.9998449',
 '59.337864,18.036587',
 '27.913011,-82.702449',
 '40.768058,-73.956599',
 '40.714056,-73.957854',
 '39.9052263,-86.0547016',
 '37.55909,-122.321836',
 '40.744618,-73.987764',
 '43.068006,-89.39256',
 '46.2121463,6.1503245',
 '41.4778641,-72.8190106',
 '48.8566667,2.3509871',
 '41.4241228,2.1634249',
 '37.3229727,-122.0385792',
 '37.270518,-121.955879',
 '37.79

In [23]:
start=time.time()
results=[]
for i in lista[:10]:   
    print(i)
    #results.append(f.nearbyAPI(i, "2000", "nightclub"))
end=time.time()
print(end - start)

40.757929,-73.985506
33.8171,-111.9035
37.566879,-122.323895
37.798853,-122.398599
34.047312,-118.445243
37.446823,-122.161523
34.0698494,-118.4054182
37.5645375,-122.3254705
37.772323,-122.214897
34.052187,-118.243425
0.002637147903442383


In [26]:
eso = f.nearbyAPI('40.757929,-73.985506', "5000", "bar")

In [27]:
len(eso)

20

In [25]:
df_molona = f.createApiDf(results)

Be aware of dropping those entries which the API return duplicated.  

In [ ]:
df_molona=df_molona.drop_duplicates()

In [ ]:
#export_csv = df_molona.to_csv('nightclubs99.csv', index=None, header=True)

# Get Coord

In [25]:
pref_comp = list(coll.find())

In [2]:
for pref in pref_comp:
    value = {"$set": {'location':f.getApiLoc(pref)}}
    coll.update_one(pref,value)

# Geoqueries

In [10]:
db, coll3 = f.connectCollection('companies','starbucks')

In [11]:
star= list(coll3.find())

In [12]:
business = list(coll2.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

In [13]:
comparatorDosPuntoCero=[]
for bu in business:
    comparatorDosPuntoCero.append(f.geoquery(bu['location'],100,coll3))

In [15]:
cuantos=[]
for i in comparatorDosPuntoCero:
    #cuantos.append(len(i))
data["starbucks 100m"]=cuantos

In [ ]:
#export_csv = data.to_csv('master99.csv', index=None, header=True)

In [ ]:
data = pd.read_csv("master99.csv") 
data.head()